### [한글 데이터셋 RNN] <hr>

In [69]:
# 모듈 로딩
import pandas as pd
from Korpora import Korpora

In [70]:
# 데이터 불러오기 및 저장
corpus = Korpora.load('nsmc')
nscmDF = pd.DataFrame(corpus.test)
nscmDF.info()


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\user\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\user

In [71]:
trainDF = nscmDF.sample(frac = 0.9, random_state = 42)
testDF = nscmDF.drop(trainDF.index)

In [111]:
# 학습용, 테스트용 데이터 분리
trainDF = nscmDF.sample(frac=0.9, random_state = 42)
trainDF.info()
print()
testDF = nscmDF.drop(trainDF.index)
testDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45000 entries, 33553 to 6838
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    45000 non-null  object
 1   label   45000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 9 to 49997
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [73]:
print(trainDF.head(5).to_markdown())
print('Training Data Size :', len(trainDF))
print('Testing Data Size :', len(testDF))

|       | text                                                                                     |   label |
|------:|:-----------------------------------------------------------------------------------------|--------:|
| 33553 | 모든 편견을 날려 버리는 가슴 따뜻한 영화. 로버트 드 니로, 필립 세이모어 호프만 영원하라. |       1 |
|  9427 | 무한 리메이크의 소재. 감독의 역량은 항상 그 자리에...                                    |       0 |
|   199 | 신날 것 없는 애니.                                                                       |       0 |
| 12447 | 잔잔 격동                                                                                |       1 |
| 39489 | 오랜만에 찾은 주말의 명화의 보석                                                         |       1 |
Training Data Size : 45000
Testing Data Size : 5000


In [74]:
from konlpy.tag import Okt
from collections import Counter

In [75]:
# 토큰화 인스턴스 생성.
tokenizer = Okt()

In [76]:
# 문장을 단어로 분리
# for text in trainDF.text :
#     print(tokenizer.morphs(text, stem=True))
#     break

train_tokens = [tokenizer.morphs(text, stem = True) for text in trainDF.text] #stem = True 하면 사전형으로 어휘사전 vocab에 등록
test_tokens = [tokenizer.morphs(text, stem = True) for text in testDF.text] 

In [77]:
print(f'[train_token] {len(train_tokens)}개\n[test_tokens] : {len(test_tokens)}개\n')
print(f'[train_token[0]] {len(train_tokens[0])}개\n[test_tokens[0]] : {len(test_tokens[0])}개\n')
print(f'[train_token[1]] {len(train_tokens[1])}개\n[test_tokens[1]] : {len(test_tokens[1])}개\n')

[train_token] 45000개
[test_tokens] : 5000개

[train_token[0]] 19개
[test_tokens[0]] : 18개

[train_token[1]] 14개
[test_tokens[1]] : 6개



In [78]:
# 단어사전 생성함수
def build_vocab(corpus, vocab_size, special_tokens):
    counter = Counter()

    # 단어/토큰에 대한 빈도수 계산
    for tokens in corpus:
        counter.update(tokens)

    # 단어/어휘 사전 생성
    vocab = special_tokens

    # 단어/어휘 사전에 빈도수가 높은 단어 추가
    for token, count in counter.most_common(vocab_size):
        vocab.append(token)
    return vocab

In [79]:
# 학습용 데이터셋 단어/어휘 사전 생성
VOCAB_SIZE = 5000
VOCAB = build_vocab(corpus = train_tokens, vocab_size = VOCAB_SIZE, special_tokens = ["<PAD>", "<UNK>"])

In [80]:
print(f'[VOCAB] ===> {len(VOCAB)}개\n{VOCAB[:30]}')

[VOCAB] ===> 5002개
['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..', '에', '가', '...', '을', '도', '들', ',', '는', '를', '은', '없다', '이다', '있다', '좋다', '?', '너무', '다', '정말', '한', '되다', '재밌다']


In [81]:
## 인코딩 : 문자 >>> 숫자로 변환
token_to_id = {voca : id for id, voca in enumerate(VOCAB)}

## 디코딩 : 숫자 >>> 문자로 변환
id_to_token = {id : voca for id, voca in enumerate(VOCAB)}

In [82]:
[[token for token in text] for text in train_tokens[0]]

[['모', '든'],
 ['편', '견'],
 ['을'],
 ['날', '다'],
 ['버', '리', '다'],
 ['가', '슴'],
 ['따', '뜻', '하', '다'],
 ['영', '화'],
 ['.'],
 ['로', '버', '트'],
 ['드'],
 ['니'],
 ['로'],
 [','],
 ['필', '립'],
 ['세', '이', '모', '어'],
 ['호', '프', '만'],
 ['영', '원', '하', '다'],
 ['.']]

[3-1] 토큰 정수화

In [83]:
# 리뷰에 문자를 정수로 변환 및 단어/어휘 사전에 없는 문자도 처리
UNK_ID = token_to_id.get('<UNK>')

train_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in train_tokens]
test_ids = [[token_to_id.get(token, UNK_ID) for token in text] for text in test_tokens]

[3-2] 데이터 구성 단어 수 맞추기 즉, 패딩(padding)
- 단어수 선정 필요
- 선정된 단어 수에 맞게 데이터 조절 => 길면 잘라내기, 짧으면 채우기

In [84]:
# 패딩 처리 함수

# - sentences : 토큰화된 문장 데이터
# - max_length : 최대 문장길이 즉, 1개 문장 구성 단어수
# - pad : 패딩 처리 시 추가될 문자 값
# - start : 패딩 시 처리 방향 [기:R 오른쪽 즉, 뒷부분 자르기/추가하기]
def pad_sequence(sentences, max_length, pad, start = 'R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[:-1*max_length]
        padd_sen = sen + [pad]*(max_length-len(sen)) if start == 'R' else [pad]*(max_length-len(sen)) + sen
        result.append(padd_sen)
    
    return result

In [85]:
# 학습용, 테스트용 데이터 패딩 처리
PAD_ID = token_to_id.get('<PAD>')
MAX_LENGTH = 32

train_ids = pad_sequence(train_ids, MAX_LENGTH, PAD_ID)
test_ids = pad_sequence(test_ids, MAX_LENGTH, PAD_ID)

In [86]:
print(f'[train ids] ===> {len(train_ids[0])}개')
print(f'[test ids] ===> {len(test_ids[0])}개')

[train ids] ===> 32개
[test ids] ===> 32개


[4] 학습 준비 <hr>
- 데이터로더 준비
- 학습용/테스트용 함수
- 모델 클래스
- 학습 관련 변수 => DEVICE, OPTIMIZER, MODEL 인스턴스, EPOCHS, BATCH SIZE, LOSS_FN

[4-1] 데이터 로더

In [87]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [88]:
dataTS.shape

torch.Size([5000, 32])

In [89]:
labelTS.shape

torch.Size([5000])

In [103]:
# 데이터셋 생성 : List를 Tensor로 변환

# 학습용 데이터셋
dataTS = torch.LongTensor(train_ids)
labelTS = torch.FloatTensor(trainDF.label.values)

print(f'dataTS => {dataTS.shape}, labelTS => {labelTS.shape}')

trainDS = TensorDataset(dataTS, labelTS)


dataTS => torch.Size([45000, 32]), labelTS => torch.Size([45000])


In [104]:
# 테스트용 데이터셋
dataTS = torch.LongTensor(test_ids)
labelTS = torch.FloatTensor(testDF.label.values)

print(f'dataTS => {dataTS.shape}, labelTS => {labelTS.shape}')

testDS = TensorDataset(dataTS, labelTS)

dataTS => torch.Size([5000, 32]), labelTS => torch.Size([5000])


In [105]:
# 데이터로더 생성
BATCH_SIZE = 32

trainDL = DataLoader(trainDS, BATCH_SIZE, shuffle = True)
testDL = DataLoader(testDS, BATCH_SIZE, shuffle = True)

[4-2] 모델 클래스
- 입력층 : Embedding Layer
- 은닉층 : RNN/LSTM Layer, Dropout Layer
- 출력층 : Linear Layer

In [106]:
from torch import nn

class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout = 0.5,
            bidirectional = True,
            model_type = 'lstm'
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings = n_vocab,
            embedding_dim = embedding_dim,
            padding_idx = 0
        )

        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True,
            )     
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True,
            )
        
        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2, 1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [107]:
from torch import optim

n_vocab = len(token_to_id)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
classifier = SentenceClassifier(n_vocab = n_vocab, hidden_dim = hidden_dim, embedding_dim = embedding_dim,
                                n_layers = n_layers).to(device)
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr = 0.001)

In [108]:
import numpy as np

def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f'Train Loss {step} : {np.mean(losses)}')

In [109]:
def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) >.5
        corrects.extend(torch.eq(yhat, labels).cpu().tolist())
    
    print(f'Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')

In [110]:
epochs = 5
interval = 500
for epoch in range(epochs):
    train(classifier, trainDL, criterion, optimizer, device, interval)
    test(classifier, testDL, criterion, device)

Train Loss 0 : 0.6988293528556824
Train Loss 500 : 0.6672000739269865
Train Loss 1000 : 0.5919460575956922
Val Loss : 0.46528638586117205, Val Accuracy : 0.7926
Train Loss 0 : 0.4705561399459839
Train Loss 500 : 0.4181335525895783
Train Loss 1000 : 0.40979841387236154
Val Loss : 0.40226447449368274, Val Accuracy : 0.8178
Train Loss 0 : 0.4062395691871643
Train Loss 500 : 0.34451214987003875
Train Loss 1000 : 0.3469868473090849
Val Loss : 0.38881009817123413, Val Accuracy : 0.8168
Train Loss 0 : 0.3715892732143402
Train Loss 500 : 0.2999616081188777
Train Loss 1000 : 0.30430211150622394
Val Loss : 0.4108718299562005, Val Accuracy : 0.827
Train Loss 0 : 0.27594271302223206
Train Loss 500 : 0.2651703634602343
Train Loss 1000 : 0.26806588622775823
Val Loss : 0.3878645946265786, Val Accuracy : 0.829
